# Inverted index title
based on assignment #3

# Setup

## General imports

The `inverted_index_colab` import requires the `inverted_index_title_colab.py` file. You should upload the file and then run this cell.

In [2]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_title_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Installing, importing, and initializing PySpark


The following code installs PySpark and its dependencies in Colab. In addition, we install GraphFrames, which is a PySpark package for dealing with graphs in a distributed fashion. Colab notebooks run on a single machine so we will work in local mode, i.e. there is no cluster of machines and both the master and worker processes run on a single machine. This will help us debug and iron out the code we will use in the second half of this assignment on an actual cluster mode in GCP. 

The installation in the next cell should take about 1 minute in a fresh environment. Don't worry about going  over the 90 seconds limit for the assignment because in our testing enviroment all of these requirements will already be met. 

In [3]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.
--2022-01-06 07:50:14--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 13.224.7.49, 13.224.7.124, 13.224.7.16, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|13.224.7.49|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ not modified on server. Omitting download.



In [4]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [5]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

## Copy some wiki data

As mentioned above, all wiki dumps were preprocessed and placed in a shared google storage bucket. To access the files in Colab, you will first have to authenticate with google storage, and then copy the data to your local environment. In GCP, the storage will already be mounted on the cluster machines and we will show you how to access it. 

In [7]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()

In [8]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


Updated property [core/project].
mkdir: cannot create directory ‘wikidumps’: File exists
Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
/ [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


# Processing wikipedia

Now that we completed the setup and have some data in our local environment, we are ready to process it using PySpark. 

## A 2-minute intro to PySpark

Let's look at our data before transforming it to RDD.

In [9]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [10]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("title", "id").rdd

## Word counts

### Term frequency

**YOUR TASK (10 POINTS):** Complete the implementation of `word_count`, which calculates the term-frequency (tf) for each word in a single document.  map function. The functions gets the text and the wiki id, calculates the tf and returns list of tuples in the form of (token, (wiki_id, tf)).

In [11]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(title, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(title.lower())]
  filtered_tokens = [tok for tok in tokens if tok not in all_stopwords]
  count = Counter(filtered_tokens)
  return [(k,(id,v)) for k,v in count.items()]
  # return [(id,(k,v)) for k,v in count.items()]

In [12]:
title = "This is a test to check the words count function actually count words"
id = 0
x = word_count(title, id)
print(x)

[('test', (0, 1)), ('check', (0, 1)), ('words', (0, 2)), ('count', (0, 2)), ('function', (0, 1)), ('actually', (0, 1))]


In [13]:
# test for word count
title = "This is a test to check the words count function actually count words"
id = 0
word_count_results = word_count(title, id)
# assert word_count_results[0][0] == 'test'
# assert len(word_count_results) == 6
# assert word_count_results[2][1][1] == word_count_results[2][1][1]
# assert type(word_count_results[0][0]) == str
# assert type(word_count_results[0][1]) == tuple


PySpark has a `map` trasformation that, just like we learned in class about Map in MapReduce, executes a given function on each element of an  RDD. It also has a `flatMap()` transformation that executes a given function on each element of an RDD, but afterwards flattens the results to create one big "list" instead multiple nested lists. For example, `flatMap` on the above `word_count` function will concatenate the tokens from multiple document into one big list rather than keeping separate list of tokens for each document like a regular `map` would produce. 

In [14]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
# word_counts.toDF().show()
word_counts.take(10)

[('anarchism', (12, 1)),
 ('autism', (25, 1)),
 ('albedo', (39, 1)),
 ('alabama', (303, 1)),
 ('achilles', (305, 1)),
 ('abraham', (307, 1)),
 ('lincoln', (307, 1)),
 ('aristotle', (308, 1)),
 ('american', (309, 1)),
 ('paris', (309, 1))]

After `flatMap` is executed, the counts for a given token are likely to be scattered accross multiple machines/workers in the cluster. In order to create the posting lists for each token, we need a reduce operation that collects them into one list and sorts them by the document id (wiki_id). 

Spark has several ways of reducing or aggergating values, but we will use `groupByKey`, which like Reduce in MapReduce, collects all the values for a given key into an iterable list of the values associated with that key. The result of `groupByKey` is an RDD with (key, value*s*) pairs. Then, we can call `map` to execute a function on each pair in that RDD or call `mapValues` to execute a function only on the values iterable.

**YOUR TASK (5 POINTS):** Complete the implementation of `reduce_word_counts`, which operates on the pairs returned by `word_count`. The functions gets a list of values (unsorted posting list) and sorts it.

In [15]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl, key = lambda tup: tup[0])

In [16]:
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# postings.toDF().show()
postings.take(50)

[('anarchism', [(12, 1)]),
 ('autism', [(25, 1)]),
 ('albedo', [(39, 1)]),
 ('alabama', [(303, 1), (1285, 1), (1286, 1), (1484, 1)]),
 ('achilles', [(305, 1), (2421, 1)]),
 ('abraham', [(307, 1), (1436, 1)]),
 ('lincoln', [(307, 1)]),
 ('aristotle', [(308, 1)]),
 ('american',
  [(309, 1),
   (615, 1),
   (659, 1),
   (771, 1),
   (863, 1),
   (869, 1),
   (966, 1),
   (1110, 1),
   (1111, 1),
   (1112, 1),
   (1201, 1),
   (1241, 1),
   (1488, 1),
   (1558, 1),
   (1890, 1),
   (1902, 1),
   (1950, 1),
   (1973, 1),
   (2011, 1),
   (2175, 1),
   (2357, 1),
   (2386, 1)]),
 ('paris', [(309, 1)]),
 ('academy', [(316, 1), (324, 1)]),
 ('award', [(316, 1)]),
 ('best', [(316, 1)]),
 ('production', [(316, 1)]),
 ('design', [(316, 1)]),
 ('awards', [(324, 1)]),
 ('actrius', [(330, 1)]),
 ('animalia', [(332, 1)]),
 ('book', [(332, 1), (1726, 1)]),
 ('international', [(334, 1)]),
 ('atomic', [(334, 1), (673, 1), (1194, 1), (1200, 1), (1206, 1), (2014, 1)]),
 ('time', [(334, 1), (1638, 1)]),
 (

In [17]:
## test reduce_word_count
token, posting_list = postings.take(1)[0]
# assert type(token) == str
# assert type(posting_list) == list
doc_id, tf = zip(*posting_list)
# assert type(doc_id[0]) == int
# assert type(tf[0]) == int
# assert np.diff(doc_id).min() > 0 
# assert np.min(tf) > 0 
# assert postings.count() == 153014

### Document frequency and filteration

In [18]:
def calculate_df(postings): #count in how many docs the word appear
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  pairs = postings.map(lambda tup: (tup[0],len(tup[1])))
  return pairs

In [19]:
x = calculate_df(postings)
x.take(2)

[('anarchism', 1), ('autism', 1)]

In [20]:

w2df = calculate_df(postings)
token, df = w2df.take(1)[0]
w2df_dict = w2df.collectAsMap()
print(w2df_dict)

{'anarchism': 1, 'autism': 1, 'albedo': 1, 'alabama': 4, 'achilles': 2, 'abraham': 2, 'lincoln': 1, 'aristotle': 1, 'american': 22, 'paris': 1, 'academy': 2, 'award': 1, 'best': 1, 'production': 1, 'design': 1, 'awards': 1, 'actrius': 1, 'animalia': 1, 'book': 2, 'international': 1, 'atomic': 6, 'time': 2, 'altruism': 1, 'ayn': 1, 'rand': 1, 'alain': 2, 'connes': 1, 'allan': 1, 'dwan': 1, 'algeria': 1, 'list': 4, 'atlas': 2, 'shrugged': 1, 'characters': 1, 'anthropology': 1, 'agricultural': 1, 'science': 1, 'alchemy': 1, 'alien': 1, 'astronomer': 2, 'ascii': 2, 'austin': 2, 'disambiguation': 18, 'animation': 1, 'apollo': 14, 'andre': 1, 'agassi': 1, 'austroasiatic': 1, 'languages': 3, 'afroasiatic': 1, 'andorra': 1, 'arithmetic': 1, 'mean': 1, 'football': 3, 'conference': 1, 'animal': 2, 'farm': 1, 'amphibian': 1, 'alaska': 1, 'agriculture': 1, 'aldous': 1, 'huxley': 1, 'ada': 3, 'aberdeen': 2, 'algae': 1, 'analysis': 3, 'variance': 2, 'alkane': 1, 'appellate': 2, 'procedure': 1, 'unit

## Partitioning and writing the index

In [21]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  inv_idx = InvertedIndex()
  p = postings.map(lambda s: (token2bucket_id(s[0]),(s[0], s[1]))).groupByKey().map(lambda x: (x[0], list(x[1]))).map(
      lambda x: inv_idx.write_a_posting_list(x))
  return p

In [22]:
x =partition_postings_and_write(postings)
x.take(2)

[defaultdict(list,
             {'abydos': [('81_000.bin', 30)],
              'agricultural': [('81_000.bin', 6)],
              'amyntas': [('81_000.bin', 48)],
              'anarchism': [('81_000.bin', 0)],
              'disabilities': [('81_000.bin', 24)],
              'mahagonny': [('81_000.bin', 66)],
              'politician': [('81_000.bin', 18)],
              'schopenhauer': [('81_000.bin', 12)],
              'titus': [('81_000.bin', 60)]}),
 defaultdict(list,
             {'abel': [('59_000.bin', 6)],
              'achaeans': [('59_000.bin', 60)],
              'anderson': [('59_000.bin', 48)],
              'andromeda': [('59_000.bin', 54)],
              'angles': [('59_000.bin', 66)],
              'angst': [('59_000.bin', 18)],
              'arachnophobia': [('59_000.bin', 36)],
              'autism': [('59_000.bin', 0)],
              'casa': [('59_000.bin', 24)],
              'serbia': [('59_000.bin', 42)]})]

The next cell uses the `collect` operator to aggregate the posting location information from the multiple buckets/indices into a single dictionary. We know in this case that the posting locations info is relatively small so this is not going to crash our master node. 

In [23]:
# test partitioning for the different buckets (10 points)
posting_locs_list = partition_postings_and_write(postings).collect()
print(posting_locs_list)
# assert len(posting_locs_list) == 124
# assert len(list(Path('.').glob('*.bin'))) == 124
pl_lens = [len(pl) for pl in posting_locs_list]
# assert np.max(pl_lens) == 141
# assert np.min(pl_lens) == 91 
# assert int(np.median(pl_lens)) == 116

[defaultdict(<class 'list'>, {'anarchism': [('81_000.bin', 0)], 'agricultural': [('81_000.bin', 6)], 'schopenhauer': [('81_000.bin', 12)], 'politician': [('81_000.bin', 18)], 'disabilities': [('81_000.bin', 24)], 'abydos': [('81_000.bin', 30)], 'amyntas': [('81_000.bin', 48)], 'titus': [('81_000.bin', 60)], 'mahagonny': [('81_000.bin', 66)]}), defaultdict(<class 'list'>, {'autism': [('59_000.bin', 0)], 'abel': [('59_000.bin', 6)], 'angst': [('59_000.bin', 18)], 'casa': [('59_000.bin', 24)], 'arachnophobia': [('59_000.bin', 36)], 'serbia': [('59_000.bin', 42)], 'anderson': [('59_000.bin', 48)], 'andromeda': [('59_000.bin', 54)], 'achaeans': [('59_000.bin', 60)], 'angles': [('59_000.bin', 66)]}), defaultdict(<class 'list'>, {'albedo': [('2_000.bin', 0)], 'angolan': [('2_000.bin', 6)], 'aluminium': [('2_000.bin', 12)], 'antiparticle': [('2_000.bin', 18)], 'acacia': [('2_000.bin', 24)], 'voting': [('2_000.bin', 30)], 'acorn': [('2_000.bin', 36)], 'fox': [('2_000.bin', 42)], 'hill': [('2_00

In [24]:
# merge the posting locations into a single dict and run more tests (5 points)
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)

# assert super_posting_locs['try'][0][0][:3] == '105'
# assert super_posting_locs['political'][0][0][:2] == '83'
# assert 'hi' not in super_posting_locs
# assert len(super_posting_locs) == 14524

In [25]:
print(super_posting_locs)

defaultdict(<class 'list'>, {'anarchism': [('81_000.bin', 0)], 'agricultural': [('81_000.bin', 6)], 'schopenhauer': [('81_000.bin', 12)], 'politician': [('81_000.bin', 18)], 'disabilities': [('81_000.bin', 24)], 'abydos': [('81_000.bin', 30)], 'amyntas': [('81_000.bin', 48)], 'titus': [('81_000.bin', 60)], 'mahagonny': [('81_000.bin', 66)], 'autism': [('59_000.bin', 0)], 'abel': [('59_000.bin', 6)], 'angst': [('59_000.bin', 18)], 'casa': [('59_000.bin', 24)], 'arachnophobia': [('59_000.bin', 36)], 'serbia': [('59_000.bin', 42)], 'anderson': [('59_000.bin', 48)], 'andromeda': [('59_000.bin', 54)], 'achaeans': [('59_000.bin', 60)], 'angles': [('59_000.bin', 66)], 'albedo': [('2_000.bin', 0)], 'angolan': [('2_000.bin', 6)], 'aluminium': [('2_000.bin', 12)], 'antiparticle': [('2_000.bin', 18)], 'acacia': [('2_000.bin', 24)], 'voting': [('2_000.bin', 30)], 'acorn': [('2_000.bin', 36)], 'fox': [('2_000.bin', 42)], 'hill': [('2_000.bin', 48)], 'alabama': [('32_000.bin', 0)], 'amphibian': [('3

Putting everything together (posting locations, df) and test that the resulting index is correct. 

In [26]:
def doc_to_title(id, title):
    return [(id,title)]

doc_title_dictionary= doc_text_pairs.flatMap(lambda x: doc_to_title(x[1],x[0]))
finall_doc_to_title = doc_title_dictionary.collectAsMap()


In [27]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
inverted.docTitle = finall_doc_to_title
# write the global stats out
inverted.write_index('.', 'title_index')